# Load Data and preprocess

In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))
import warnings
warnings.filterwarnings('ignore')

In [2]:
#pip install ucimlrepo

In [3]:
from src.data.load_data import load_input_data
from src.data.load_data import preprocess_data
#from src.data.load_data import preprocess_data_no_onehot

In [4]:
X_train, X_test, y_train, y_test, metadata, variables, datasetdf = load_input_data()

In [5]:
import pandas as pd
from src.data.load_data import preprocess_num

num_cols = ['Attribute2', 'Attribute5', 'Attribute8', 'Attribute11',
            'Attribute13', 'Attribute16', 'Attribute18']

cat_cols = X_train.columns.difference(num_cols)

X_train_raw,X_test_raw = X_train,X_test
#X_train, X_test, datasetdf = preprocess_data(X_train, X_test, datasetdf, num_cols)
X_train_num, X_test_num, datasetdf_num = preprocess_num(X_train, X_test, datasetdf, num_cols)
X_train, X_test, datasetdf = preprocess_data(X_train, X_test, datasetdf, num_cols)

#pd.set_option('display.max_columns', None)
#print(X_train.head())
print(datasetdf.head())

   Attribute2  Attribute5  Attribute8  Attribute11  Attribute13  Attribute16  \
0   -1.235427   -0.760138    0.945404     1.076342     2.880568     1.074000   
1    2.277975    1.028842   -0.835976    -0.747967    -1.230428    -0.724565   
2   -0.733512   -0.413341   -0.835976     0.164188     1.236170    -0.724565   
3    1.776060    1.751243   -0.835976     1.076342     0.870748    -0.724565   
4    0.270317    0.624432    0.054714     1.076342     1.601592     1.074000   

   Attribute18  class  Attribute1_A12  Attribute1_A13  ...  Attribute12_A124  \
0    -0.434114      1           False           False  ...             False   
1    -0.434114      2            True           False  ...             False   
2     2.303542      1           False           False  ...             False   
3     2.303542      1           False           False  ...             False   
4     2.303542      2           False           False  ...              True   

   Attribute14_A142  Attribute14_A143 

# Train model

In [6]:
from src.model.train_model import train_classifier
from src.model.train_model import train_classifier_LR

In [7]:
#clf = train_classifier(X_train,y_train,n_estimators=100, random_state=42) # this is for random forrest
clf = train_classifier_LR(X_train,y_train, random_state=42)

# Test model

In [8]:
from src.model.test_model import test_classifier_prediction

In [9]:
y_pred,classification_report,accuracy_score = test_classifier_prediction(clf,X_test,y_test)

In [10]:
print(accuracy_score)

0.78


In [11]:
print(classification_report)

              precision    recall  f1-score   support

           1       0.81      0.89      0.85       210
           2       0.67      0.52      0.59        90

    accuracy                           0.78       300
   macro avg       0.74      0.71      0.72       300
weighted avg       0.77      0.78      0.77       300



### User constraints / Ontology constraints

In [12]:
from src.ontology.generate_rules import *

In [13]:
X_train_positive = X_train_num[y_train==1]
X_train_negative = X_train_num[y_train==2]

In [14]:
positive_profile = summarize_features(X_train_positive)

In [15]:
#positive_profile

In [16]:
negative_profile = summarize_features(X_train_negative)

In [17]:
#negative_profile

In [18]:
#p_statistical = extract_statistical_rules(X_train_positive, X_train_raw, num_quantiles=(0.1, 0.9), cat_threshold=0.85)
p_statistical = extract_statistical_rules(X_train_positive, X_train_num, num_quantiles=(0.1, 0.9), cat_threshold=0.85)

In [19]:
p_statistical

{'Attribute2': (-1.235, 1.274),
 'Attribute5': (-0.85, 1.224),
 'Attribute8': (-1.727, 0.945),
 'Attribute10': 'A101',
 'Attribute11': (-1.66, 1.076),
 'Attribute13': (-1.048, 1.419),
 'Attribute16': (-0.725, 1.074),
 'Attribute18': (-0.434, 2.304)}

In [20]:
p_permutation_imp_top_features = top_features(clf, X_train, y_train, n_repeats=5, random_state=42)

In [21]:
p_permutation_imp_top_features

['Attribute1_A14',
 'Attribute9_A93',
 'Attribute6_A65',
 'Attribute2',
 'Attribute12_A124',
 'Attribute3_A34',
 'Attribute8',
 'Attribute15_A153',
 'Attribute14_A143',
 'Attribute7_A74',
 'Attribute15_A152',
 'Attribute1_A12',
 'Attribute5']

In [22]:
p = combined_features(p_permutation_imp_top_features,p_statistical)

In [23]:
p

{'Attribute2': (-1.235, 1.274),
 'Attribute5': (-0.85, 1.224),
 'Attribute8': (-1.727, 0.945),
 'Attribute10': 'A101',
 'Attribute11': (-1.66, 1.076),
 'Attribute13': (-1.048, 1.419),
 'Attribute16': (-0.725, 1.074),
 'Attribute18': (-0.434, 2.304),
 'Attribute1': 'A14',
 'Attribute9': 'A93',
 'Attribute6': 'A65',
 'Attribute12': 'A124',
 'Attribute3': 'A34',
 'Attribute15': 'A153',
 'Attribute14': 'A143',
 'Attribute7': 'A74'}

In [24]:
p_num = {key: value for key, value in p.items() if isinstance(value, tuple)}
p_cat = {key: value for key, value in p.items() if isinstance(value, str)}
print(p_num)
print(p_cat)

{'Attribute2': (-1.235, 1.274), 'Attribute5': (-0.85, 1.224), 'Attribute8': (-1.727, 0.945), 'Attribute11': (-1.66, 1.076), 'Attribute13': (-1.048, 1.419), 'Attribute16': (-0.725, 1.074), 'Attribute18': (-0.434, 2.304)}
{'Attribute10': 'A101', 'Attribute1': 'A14', 'Attribute9': 'A93', 'Attribute6': 'A65', 'Attribute12': 'A124', 'Attribute3': 'A34', 'Attribute15': 'A153', 'Attribute14': 'A143', 'Attribute7': 'A74'}


### Final outputs of Input section of pipeline
For experimentation, here are some values

In [25]:
from src.utilities.desired_space import desired_space
X_test_negative = X_test[y_test==2]
x= X_test_negative[0:1] #test instance #select first row as dataFrame
print(clf.predict(x))
p_num = p_num
p_cat = p_cat
#p= {
    #'Attribute5':(0, 1) #credit amount #I put a normalized amount here
#}#perturbation map
#desired_space= desired_space(2, datasetdf) #all the feature vectors with output=2 #not preprocessed! Potential issue from authors!
#print(datasetdf.head())
desired_space = desired_space(1,datasetdf) #did preprocessing also to datasetdf
#print(X_train.head())
#print(desired_space.head())
cat_f= X_train.select_dtypes(include=['bool']) #categorical features
#print(cat_f)
num_f= X_train.select_dtypes(include=['int64', 'float64']) #numerical features
#print(num_f)
protect_f= ['Attribute9_A92'  'Attribute9_A93'  'Attribute9_A94', 'Attribute5'] # Personal status and sex (X_train.head() shows only 3 values (columns) while there are 5) #protected features
features= X_train.columns #all features
t= 1 #desired outcome
f= clf #black-box model for which we want to find CF
X= X_train #training data
step = {attribute:0.01 for attribute in p_num.keys()}
#step= {
#    'Attribute5': 0.01 #credit amount steps #tiny amount bc of normalized feature values
#} #dictionary holding the feature distribution to be used for single feature perturbation method

[2]


### Derived variables

In [26]:
f2change = p.keys()

# UFCE

### Feature Selection: Mutual Information (MI)

In [27]:
from itertools import combinations
from src.utilities.mutual_information import mi_score
from src.utilities.mutual_information import MI
feature_pairs = list(combinations(list(features), 2))
print(feature_pairs[:5])
mi_list = MI(feature_pairs, X_train) #used all features available #original authors do over entire X but it is not preprocessed... Which i think is nessecary? #also works for bool one-hot encoding 
# top 5 
print(mi_list[:5])

[('Attribute2', 'Attribute5'), ('Attribute2', 'Attribute8'), ('Attribute2', 'Attribute11'), ('Attribute2', 'Attribute13'), ('Attribute2', 'Attribute16')]
[('Attribute9_A92', 'Attribute9_A93'), ('Attribute2', 'Attribute5'), ('Attribute17_A172', 'Attribute17_A173'), ('Attribute12_A124', 'Attribute15_A153'), ('Attribute3_A32', 'Attribute3_A34')]


### Nearest Neighbour: KD Tree

In [28]:
from src.utilities.nearest_neighbours import FNN
nn = FNN(desired_space,x,3.5) #radius arbitrarily set; can experiment with how close we want it to be to x #3.5 for euclidean distance is around 0.5 for each of the 48 features difference from x, get 3 datapoints
#print(nn)

### Subspace: Intervals

In [29]:
from src.utilities.nearest_neighbours import intervals
from src.utilities.nearest_neighbours import intervals2
#subspace = intervals(nn,p,f2change,x) 
subspace = intervals2(nn,p_num,p_cat,f2change,x)
#print(subspace) #correct output

### CF Generator: Single Feature

In [30]:
from src.utilities.UFCE import SF
#cf = SF(x,X,cat_f,p,f,t,step)
cf = SF(x,X_train,p_num,p_cat,f,t,step) # changed from X_train_raw to X_train
print(cf)

     Attribute2  Attribute5  Attribute8  Attribute11  Attribute13  \
677    4.285632     0.89566   -0.835976    -0.747967    -1.047717   

     Attribute16  Attribute18  Attribute1_A12  Attribute1_A13  Attribute1_A14  \
677    -0.724565    -0.434114               0               0            True   

     ...  Attribute12_A124  Attribute14_A142  Attribute14_A143  \
677  ...             False             False              True   

     Attribute15_A152  Attribute15_A153  Attribute17_A172  Attribute17_A173  \
677              True             False             False              True   

     Attribute17_A174  Attribute19_A192  Attribute20_A202  
677             False             False             False  

[1 rows x 48 columns]


### CF Generator: Double Features

In [31]:
from src.utilities.UFCE import DF
z = DF(datasetdf,x,X_train,subspace,mi_list,cat_f,num_f,features,protect_f,f,t)

In [32]:
# print("original: ")
# print(x)
# print("counterfactual: ")
# print(z)

In [33]:
unique_cf_signatures = set()
unique_counterfactuals = []

for cf in z:
    # Align columns
    common_cols = x.columns.intersection(cf.columns)
    x_common = x[common_cols]
    cf_common = cf[common_cols]
    
    # Find differences
    differences = (x_common != cf_common).iloc[0]
    
    # Represent difference as a tuple of (feature, original_val, cf_val)
    diff_signature = tuple(
        (col, x[col].values[0], cf[col].values[0])
        for col in common_cols[differences]
    )
    
    # Add only if this signature is new
    if diff_signature not in unique_cf_signatures:
        unique_cf_signatures.add(diff_signature)
        unique_counterfactuals.append(cf)


In [34]:
print("Comparing original instance with counterfactuals:")

for i, cf in enumerate(unique_counterfactuals):
    print(f"\n--- Counterfactual {i+1} ---")
    print(clf.predict(cf))
    
    # Find common columns
    common_cols = x.columns.intersection(cf.columns)
    x_common = x[common_cols]
    cf_common = cf[common_cols]
    
    # Compare
    differences = (x_common != cf_common).iloc[0]
    
    if differences.any():
        for col in common_cols[differences]:
            print(f"{col}: original = {x[col].values[0]}, counterfactual = {cf[col].values[0]}")
    else:
        print("No differences from original.")


Comparing original instance with counterfactuals:

--- Counterfactual 1 ---
[1]
Attribute2: original = 4.285632296481258, counterfactual = 1.77
Attribute11: original = -0.7479667044950186, counterfactual = -0.6313244709548823


### Evaluation

In [35]:
X_train_negative

,Attribute1,Attribute2,Attribute3,Attribute4,Attribute5,Attribute6,Attribute7,Attribute8,Attribute9,Attribute10,Attribute11,Attribute12,Attribute13,Attribute14,Attribute15,Attribute16,Attribute17,Attribute18,Attribute19,Attribute20
10,A12,-0.733512,A32,A40,-0.713001,A61,A72,0.054714,A92,A101,-1.660121,A123,-0.956361,A143,A151,-0.724565,A173,-0.434114,A191,A201
827,A14,-0.231598,A30,A49,0.360687,A61,A73,-0.835976,A93,A101,-0.747967,A123,0.048549,A142,A152,1.074000,A173,2.303542,A191,A201
812,A11,1.274146,A34,A41,2.404809,A61,A74,0.945404,A93,A101,1.076342,A123,-1.047717,A143,A152,1.074000,A173,-0.434114,A192,A201
414,A11,0.270317,A32,A40,-0.680828,A65,A73,0.945404,A92,A101,-0.747967,A122,-0.042807,A143,A152,-0.724565,A173,-0.434114,A191,A201
983,A11,1.274146,A32,A41,1.881059,A61,A73,-0.835976,A93,A101,-0.747967,A122,-0.865006,A143,A152,-0.724565,A173,2.303542,A191,A201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
963,A14,0.270317,A32,A43,-0.300735,A63,A75,0.054714,A93,A101,-0.747967,A123,-0.042807,A141,A152,1.074000,A173,-0.434114,A192,A201
648,A13,0.270317,A32,A40,-0.843190,A61,A74,0.945404,A93,A101,0.164188,A124,0.231260,A141,A153,-0.724565,A173,2.303542,A191,A201
131,A11,1.274146,A33,A46,1.379007,A61,A73,0.945404,A93,A101,0.164188,A122,-0.590939,A142,A152,-0.724565,A173,-0.434114,A192,A201
603,A14,1.274146,A32,A42,0.055415,A61,A73,0.945404,A92,A101,-0.747967,A123,-0.682295,A143,A152,-0.724565,A174,-0.434114,A192,A201


In [36]:
from src.evaluation.metrics import *

In [37]:
actionable_features = X_test_negative[0:1].columns
feasibility_score = proximity_score_num = proximity_score_cat = sparsity_score = 0.0
cf_counter_SF = 0
for idx, row in X_test_negative.iterrows():
    x = row.to_frame().T
    nn = FNN(desired_space,x,3.5)
    subspace = intervals2(nn,p_num,p_cat,f2change,x)
    cf_SF = SF(x,X_train,p_num,p_cat,f,t,step)
    if(len(cf_SF)!=0):
        compute_feasibility(x, cf_SF, actionable_features, clf, 1, X_train,0.5)
        if compute_feasibility(x, cf_SF, actionable_features, clf, 1, X_train,0.5):
            feasibility_score = feasibility_score + 1
        
        sparsity_score = sparsity_score + compute_sparsity(x, cf_SF)
        cat_cols = list(x.columns.difference(num_cols))
        prox_num,prox_cat = compute_proximity(x, cf_SF, num_cols,cat_cols)
        proximity_score_num = proximity_score_num + prox_num
        proximity_score_cat = proximity_score_cat + prox_cat
        print("Evaluation for SF done for: ",idx)
        cf_counter_SF += 1
    else:
        print("Empty CF for: ",idx)

Evaluation for SF done for:  677
Evaluation for SF done for:  235
Evaluation for SF done for:  375
Empty CF for:  602
Empty CF for:  739
Evaluation for SF done for:  840
Evaluation for SF done for:  722
Evaluation for SF done for:  510
Evaluation for SF done for:  301
Evaluation for SF done for:  796
Evaluation for SF done for:  578
Evaluation for SF done for:  188
Evaluation for SF done for:  900
Evaluation for SF done for:  548
Evaluation for SF done for:  751
Evaluation for SF done for:  598
Empty CF for:  332
Evaluation for SF done for:  614
Empty CF for:  11
Evaluation for SF done for:  915
Evaluation for SF done for:  790
Evaluation for SF done for:  129
Evaluation for SF done for:  182
Evaluation for SF done for:  368
Evaluation for SF done for:  116
Evaluation for SF done for:  435
Evaluation for SF done for:  199
Evaluation for SF done for:  948
Evaluation for SF done for:  308
Evaluation for SF done for:  953
Evaluation for SF done for:  228
Evaluation for SF done for:  291
E

In [38]:
print(feasibility_score)
print(proximity_score_num)
print(proximity_score_cat)
print(sparsity_score)

45.0
0.0
74.895202020202
1.3333333333333335


In [39]:
avg_feasibility = feasibility_score / cf_counter_SF
print(avg_feasibility)

0.5487804878048781


In [40]:
avg_proximity_num = proximity_score_num / cf_counter_SF
print(avg_proximity_num)

0.0


In [41]:
avg_proximity_cat = proximity_score_cat / cf_counter_SF
print(avg_proximity_cat)

0.9133561221975854


In [42]:
avg_sparsity = sparsity_score / cf_counter_SF
print(avg_sparsity)

0.016260162601626018


In [55]:
cf_counter_SF

82

In [43]:
actionable_features = X_test_negative[0:1].columns
feasibility_score_DF = proximity_score_num_DF = proximity_score_cat_DF = sparsity_score_DF = 0.0
cf_counter_DF = 0
for idx, row in X_test_negative.iterrows():
    x = row.to_frame().T
    nn = FNN(desired_space,x,3.5)
    subspace = intervals2(nn,p_num,p_cat,f2change,x)

    cf_DF = DF(datasetdf,x,X_train,subspace,mi_list,cat_f,num_f,features,protect_f,f,t)
    unique_cf_signatures = set()
    unique_counterfactuals = []
    for cf in cf_DF:
        # Align columns
        common_cols = x.columns.intersection(cf.columns)
        x_common = x[common_cols]
        cf_common = cf[common_cols]
        
        # Find differences
        differences = (x_common != cf_common).iloc[0]
        
        # Represent difference as a tuple of (feature, original_val, cf_val)
        diff_signature = tuple(
            (col, x[col].values[0], cf[col].values[0])
            for col in common_cols[differences]
        )
        
        # Add only if this signature is new
        if diff_signature not in unique_cf_signatures:
            unique_cf_signatures.add(diff_signature)
            unique_counterfactuals.append(cf)
    best_cf = None
    min_total_proximity = float('inf')
    
    for cf in unique_counterfactuals:
        if len(cf) != 0:
            # Compute proximity for current CF
            cat_cols = list(x.columns.difference(num_cols))
            prox_num, prox_cat = compute_proximity(x, cf, num_cols, cat_cols)
            total_proximity = prox_num + prox_cat
    
            # Keep the CF with the lowest proximity
            if total_proximity < min_total_proximity:
                min_total_proximity = total_proximity
                best_cf = cf
        else:
            print("Empty CF for: ", idx)
    
    # Evaluate only the best CF
    if best_cf is not None:
        print("Best CF selected for: ", idx)
    
        # Feasibility
        if compute_feasibility(x, best_cf, actionable_features, clf, 1, X_train, 0.5):
            feasibility_score_DF += 1
    
        # Sparsity
        sparsity_score_DF += compute_sparsity(x, best_cf)
    
        # Proximity
        prox_num, prox_cat = compute_proximity(x, best_cf, num_cols, cat_cols)
        proximity_score_num_DF += prox_num
        proximity_score_cat_DF += prox_cat
    
        cf_counter_DF += 1
    else:
        print("No valid CF found for: ", idx)

    # for cf in unique_counterfactuals:
    #     if(len(cf)!=0):
    #         compute_feasibility(x, cf, actionable_features, clf, 1, X_train,0.5)
    #         if compute_feasibility(x, cf, actionable_features, clf, 1, X_train,0.5):
    #             feasibility_score_DF = feasibility_score_DF + 1
            
    #         sparsity_score_DF = sparsity_score_DF + compute_sparsity(x, cf)
    #         cat_cols = list(x.columns.difference(num_cols))
    #         prox_num,prox_cat = compute_proximity(x, cf, num_cols,cat_cols)
    #         proximity_score_num_DF = proximity_score_num_DF + prox_num
    #         proximity_score_cat_DF = proximity_score_cat_DF + prox_cat
    #         print("Evaluation for DF done for: ",idx)
    #         cf_counter_DF += 1
    #     else:
    #         print("Empty CF for: ",idx)
    

Best CF selected for:  677
Best CF selected for:  235
Best CF selected for:  375
No valid CF found for:  602
No valid CF found for:  739
Best CF selected for:  840
Best CF selected for:  722
Best CF selected for:  510
Best CF selected for:  301
Best CF selected for:  796
Best CF selected for:  578
Best CF selected for:  188
Best CF selected for:  900
No valid CF found for:  548
Best CF selected for:  751
Best CF selected for:  598
No valid CF found for:  332
Best CF selected for:  614
No valid CF found for:  11
Best CF selected for:  915
Best CF selected for:  790
Best CF selected for:  129
Best CF selected for:  182
Best CF selected for:  368
Best CF selected for:  116
Best CF selected for:  435
Best CF selected for:  199
Best CF selected for:  948
Best CF selected for:  308
Best CF selected for:  953
Best CF selected for:  228
Best CF selected for:  291
Best CF selected for:  475
Best CF selected for:  846
Best CF selected for:  633
Best CF selected for:  192
Best CF selected for:  3

In [44]:
print(feasibility_score_DF)
print(proximity_score_num_DF)
print(proximity_score_cat_DF)
print(sparsity_score_DF)

74.0
62.65160637524668
74.0
3.083333333333331


In [45]:
avg_feasibilit_DF = feasibility_score_DF / cf_counter_DF
print(avg_feasibilit_DF)

1.0


In [46]:
avg_proximity_num_DF = proximity_score_num_DF / cf_counter_DF
print(avg_proximity_num_DF)

0.8466433293952254


In [47]:
avg_proximity_cat_DF = proximity_score_cat_DF / cf_counter_DF
print(avg_proximity_cat_DF)

1.0


In [48]:
avg_sparsity_DF = sparsity_score_DF / cf_counter_DF
print(avg_sparsity_DF)

0.04166666666666663


In [49]:
cf_counter_DF

74

In [50]:
cf_feasibility = {}
cf_sparsity = {}
cf_proximity_num = {}
cf_proximity_cat = {}

x = X_test_negative[5:6]
nn = FNN(desired_space,x,3.5)
subspace = intervals2(nn,p_num,p_cat,f2change,x)
cf_DF = DF(datasetdf,x,X_train,subspace,mi_list,cat_f,num_f,features,protect_f,f,t)
unique_cf_signatures = set()
unique_counterfactuals = []
for cf in cf_DF:
    # Align columns
    common_cols = x.columns.intersection(cf.columns)
    x_common = x[common_cols]
    cf_common = cf[common_cols]
    
    # Find differences
    differences = (x_common != cf_common).iloc[0]
    
    # Represent difference as a tuple of (feature, original_val, cf_val)
    diff_signature = tuple(
        (col, x[col].values[0], cf[col].values[0])
        for col in common_cols[differences]
    )
    
    # Add only if this signature is new
    if diff_signature not in unique_cf_signatures:
        unique_cf_signatures.add(diff_signature)
        unique_counterfactuals.append(cf)
for idx, cf in enumerate(unique_counterfactuals):
    if(len(cf)!=0):
        compute_feasibility(x, cf, actionable_features, clf, 1, X_train,0.5)
        if compute_feasibility(x, cf, actionable_features, clf, 1, X_train,0.5):
            cf_feasibility[idx] = 1
        else:
            cf_feasibility[idx] = 0
        cf_sparsity[idx] = compute_sparsity(x, cf)
        cat_cols = list(x.columns.difference(num_cols))
        prox_num,prox_cat = compute_proximity(x, cf, num_cols,cat_cols)
        cf_proximity_num[idx] = prox_num
        cf_proximity_cat[idx] = prox_cat

In [51]:
cf_feasibility

{0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1}

In [52]:
cf_sparsity

{0: 0.041666666666666664,
 1: 0.041666666666666664,
 2: 0.041666666666666664,
 3: 0.041666666666666664,
 4: 0.041666666666666664,
 5: 0.041666666666666664}

In [53]:
cf_proximity_num

{0: 0.4407686454671224,
 1: 0.28461785049109867,
 2: 1.6881257846340842,
 3: 1.1013971093891854,
 4: 1.3470552276506973,
 5: 0.8058118189121207}

In [54]:
cf_proximity_cat

{0: 1.0, 1: 1.0, 2: 1.0, 3: 1.0, 4: 1.0, 5: 1.0}